In [1]:
import numpy as np
import random 
from collections import defaultdict
from CA import *

In [4]:
def main_loop(P_init, 
              iterations=10, road_len=int(500), Vmax=6, Vrandom=True,
             p_gen=0.1):
    
    # make the initial road and cars
    cars, road = random_init_cars(road_len, P_init,
                                  Vrandom,
                                  Vmax)
    # make initial car positions
    positions = car_positions(road)
    
    # run the main experiment loop
    for i in range(iterations):
        # update the position using current speed
        cars, road = position_update(positions, cars, road)
        
        # generate randomly new cars at the beginning of the road
        #cars, road = generate_new_cars(cars, road, p_gen=0.5, speed_random=False, Vmax=6)
        
        # get the new car positions from the new array
        positions = car_positions(road)
        
        # update the speed based on the new position
        cars = speed_update(positions, cars, road, Vmax)
        print(road)

        
    return cars, road     
        
cars, road = main_loop(0.3, 200)   
print(road[:])

[  1   0   0   2   0   0   0   0   0   0   0   0   3   0   4   0   0   5
   0   0   0   0   6   0   0   0   0   0   0   7   0   0   0   0   0   0
   8   0   9   0   0   0   0   0  10   0   0   0   0   0   0   0   0   0
   0   0  11   0   0   0   0   0  12   0   0   0   0  13   0   0  14   0
  15   0   0   0   0   0   0  16   0   0   0   0   0   0   0   0   0   0
   0   0   0  17   0   0  18   0   0   0   0   0   0  19   0   0   0  20
   0   0   0  21   0  22   0   0   0  23   0   0  24   0  25   0   0  26
   0   0  27   0   0   0  28   0   0  29   0   0   0  30   0   0   0   0
  31   0   0  32   0   0   0   0  33   0  34   0   0   0   0   0  35   0
   0   0   0   0   0  36   0   0   0  37   0   0  38   0  39   0  40   0
   0   0   0   0  41   0   0   0   0   0  42   0  43   0   0  44   0   0
   0  45   0   0  46   0   0   0   0   0  47   0   0   0   0  48   0   0
   0  49   0   0   0   0   0   0   0  50   0   0   0  51   0   0   0  52
   0   0   0   0   0   0   0  53   0   0   0  54   